# Imports

In [165]:
# Utils
from datetime import datetime
from datetime import timedelta
import os
import sys
import time
from itertools import combinations

# Data management
import numpy as np
import pandas as pd

# Data fetching
import yfinance as yf

# Spread generation
from sklearn.linear_model import LinearRegression

# Utils

## Stonk price data download

### Input ticker names by industry

In [23]:
def get_tickers_by_industry(industries=None, data_dir=None, filename=None):
    '''
    Read the CSV file containing all tickers and their subindustries and return tickers from the selected subindustries in a list.
    
    -Args:
        industries (List(string)): if not given, return all tickers; else the list can contain:
            'technology_hardware_and_equipment'
            'software_and_services'
            'media_and_entertainment'
            'retailing'
            'automobiles_and_components'
            'semiconductors_and_semiconductor_equipment'
            'health_care_equipment_and_services'
            'banks'
            'pharmaceuticals_biotechnology_and_life_sciences'
            'food_and_staples_retailing'
            'oil_gas_and_consumable_fuels'
            'food_beverage_and_tobacco'
            'telecommunication_services'
            'consumer_durables_and_apparel'
            'consumer_services'
            'transportation'
            'diversified_financials'
            'utilities'
            'capital_goods'
            'insurance'
            'chemicals'
            'metals_and_mining'
            'commercial_and_professional_services'
            'containers_and_packaging'
            'energy_equipment_and_services'
            'construction_materials'
            'paper_and_forest_products'
    
    -Returns:
        tickers (pandas Series): list of selected ticker names
    '''
    filename = 'stonk_list.csv' if filename is None else filename
    data_dir = 'data' if data_dir is None else data_dir
    
    path_to_csv = os.path.join(data_dir, filename)
    stonk_list = pd.read_csv(path_to_csv)
    return stonk_list.set_index('ticker') if industries is None else stonk_list[stonk_list['subindustry'].isin(industries)].set_index('ticker')

In [63]:
def download_stonk_prices(stonk_list, period_years=3, date_from=None, date_to=None, interval='1d', source='yfinance', data_dir='data', file_prefix='stonks', proxy=False):    
    '''
    Returns historical price data for the selected stonks.

    -Args:
        stonk_list (List(string)): List of stonk identifiers as strings, case unsensitive
        period_years (float): How many years of data to download until date_to, can be a floating point number
    -Optional:
        date_from (datetime): Start date for stonk data (use instead of period_years)
        date_to (datetime): End date for stonk data
        interval (string): Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        source (string): Where to source data from. Valid sources: yfinance
        data_dir (string): Folder name where to output downloaded data
        file_prefix (string): Prefix of CSV file containing downloaded data inside data_dir
        proxy (boolean): Whether to use a proxy connection to avoid API limits/blocks
                
    -Returns:
        stonks (Pandas Dataframe): Pandas Dataframe containing requested ticker prices
    '''
    
    date_to = datetime.now() if date_to is None else date_to
    date_from = date_to-(timedelta(days=int(365*period_years))) if date_from is None else date_from
    
    if source.lower() == 'yfinance':
        stonks = yf.download(list(stonk_list), start=date_from, end=date_to, interval=interval, group_by='column', threads=True, rounding=True)['Adj Close']
        stonks.dropna(axis=0, how='all', inplace=True)
        stonks.sort_values(by='Date', inplace=True)
    else:
        raise ValueError('Unsupported data source')
        
    from_date_string = stonks.index[0].strftime('%Y-%m-%d')
    to_date_string = stonks.index[-1].strftime('%Y-%m-%d')
    
    filename = '{prefix}_{from_date}_to_{to_date}.csv'.format(prefix=file_prefix, from_date=from_date_string, to_date=to_date_string)
    file_path = os.path.join(data_dir, filename)
    
    stonks.to_csv(path_or_buf=file_path, header=True, index=True, na_rep='NaN')
    
    return stonks

### Quick examples

In [53]:
# Gets all ticker names (no argument given)
ticker_list = get_tickers_by_industry()

In [62]:
ticker_list

,subindustry
ticker,
AAPL,technology_hardware_and_equipment
MSFT,software_and_services
GOOG,media_and_entertainment
AMZN,retailing
TSLA,automobiles_and_components
...,...
WSBCP,banks
WSO-B,capital_goods
WTFCP,banks


In [64]:
# Download ticker price data for the tickers selected above (saved to .csv automatically)
df = download_stonk_prices(ticker_list.index, period_years=4)

[*********************100%***********************]  2283 of 2283 completed

9 Failed downloads:
- WFC PRN: No data found, symbol may be delisted
- WCC-PA: No data found for this date range, symbol may be delisted
- ET-PC: No data found for this date range, symbol may be delisted
- NRZ-PD: No data found for this date range, symbol may be delisted
- FHN PRA: No data found, symbol may be delisted
- SNX.VI: No data found, symbol may be delisted
- ALL-PB: No data found for this date range, symbol may be delisted
- ET-PD: No data found for this date range, symbol may be delisted
- RXN.VI: No data found, symbol may be delisted


In [10]:
def preprocess_stock_list(raw_data_path='data/raw_stonk_list.xls', output_path='data/stonk_list.csv'):
    '''
    Parses a raw excel file from CapitalIQ containing ticker names and their subindustries, validates
    unusual ticker names with Yahoo Finance, saving the processed data in CSV format.

        Parameters:
            Required:
                raw_data_path (string):
                    Path to the raw excel file.
                output_path (string):
                    Path where to save the parsed data.
                
        Returns:
            Nothing
    '''
    
    df = pd.read_excel(io=raw_data_path)
    
    # Drop NA rows
    df.dropna(axis=0, inplace=True)
    
    # Reset index and drop the first row
    df.reset_index(inplace=True, drop=True)
    df.drop(index=0, axis=0, inplace=True)
    
    # Drop unwanted columns
    df.drop(columns=df.columns[[1, 2, 3, 4, 5, 7, 8, 9]], inplace=True)
    
    # Rename remaining columns
    df.columns = ['ticker', 'subindustry']
    
    # Remove the '(Primary)' tag from subindustries
    df['subindustry'] = df['subindustry'].str.replace(r' \(Primary\)', '')
    
    # Remove everything until (and including) the semicolon for tickers
    df['ticker'] = df['ticker'].str.replace(r'(.*:)', '')
    
    df['ticker'] = df['ticker'].str.replace(r' WI', '.VI')
    df['ticker'] = df['ticker'].str.replace(r'\.WI', '.VI')
    
    # Replace the ticker endings for a Yahoo finance supported format
    df['ticker'] = df['ticker'].str.replace(r'\.PR', '-P')
    # df['ticker'] = df['ticker'].str.replace(r' PR', '-P')
    
    # Take all remaining tickers that have a dot
    dotted = df[df['ticker'].str.fullmatch(r'[A-Z]*\.[A-Z]')]
    
    # Replace the dots with dashes
    dashed = dotted.copy()
    dashed['ticker'] = dashed['ticker'].str.replace(r'\.', '-')
    
    # Remove the dots
    undotted = dotted.copy()
    undotted['ticker'] = undotted['ticker'].str.replace(r'\.', '')

    # Combine all variantas together
    all_variants = pd.concat([dotted, dashed, undotted])
    
    # Run all of these through Yahoo finance, get last day's price
    stonks = yf.download(list(all_variants['ticker'].astype('string').values), period='1m', interval='1d', group_by='column')
    
    # Drop all NA tickers (that failed to download)
    valid_tickers = stonks['Adj Close'].iloc[-1].dropna(axis=0).to_frame().reset_index()
    
    # Rename columns
    valid_tickers.columns = ['ticker', 'price']
    
    # Add subindustries to the remaining valid tickers
    valid_tickers = valid_tickers.join(all_variants.set_index('ticker'), on='ticker')
    
    # Drop the price column
    valid_tickers.drop(columns=valid_tickers.columns[[1]], inplace=True)
    
    # Remove all tickers that have a dot from main dataframe
    df = df[~df['ticker'].str.fullmatch(r'[A-Z]*\.[A-Z]')]
    
    # Add the validated tickers back
    df = pd.concat([df, valid_tickers], axis=0, ignore_index=True)
    
    # Make the subindustry strings more code friendly
    df['subindustry'] = df['subindustry'].str.replace(' ', '_')
    df['subindustry'] = df['subindustry'].str.lower()
    df['subindustry'] = df['subindustry'].str.replace(',', '')
    
    df.to_csv(path_or_buf=output_path, header=True, index=False)

## Stock price data input

In [85]:
def read_stonk_data(date_from, date_to, data_prefix=None, data_dir=None):
    data_dir = 'data' if data_dir is None else data_dir
    data_prefix = 'stonks' if data_prefix is None else data_prefix
    
    path = os.path.join(data_dir, '{}_{}_to_{}.csv'.format(data_prefix, date_from, date_to))
    stonks = pd.read_csv(path)
    stonks['Date'] = pd.to_datetime(stonks['Date']).dt.date
    stonks = stonks.set_index('Date')
    
    stonks.dropna(axis=1, how='all', thresh=len(stonks) * 0.95, inplace=True)
    stonks.dropna(axis=0, how='all', thresh=len(stonks) * 0.95, inplace=True)
    stonks.fillna(axis=1, method='ffill', inplace=True)
    stonks.dropna(axis=1, how='any', inplace=True)
    
    assert stonks.isna().sum().sum() == 0
    
    return stonks.T

In [92]:
def get_stonk_data_by_industry(date_from, date_to, industries=None, data_prefix=None, stonk_list_filename=None, data_dir=None):
    '''
    Read the CSV file containing all stonk price data and return the tickers from the selected subindustries.
    
    -Args:
        industries (List(string)): if not given, return all tickers; else the list can contain:
            'technology_hardware_and_equipment'
            'software_and_services'
            'media_and_entertainment'
            'retailing'
            'automobiles_and_components'
            'semiconductors_and_semiconductor_equipment'
            'health_care_equipment_and_services'
            'banks'
            'pharmaceuticals_biotechnology_and_life_sciences'
            'food_and_staples_retailing'
            'oil_gas_and_consumable_fuels'
            'food_beverage_and_tobacco'
            'telecommunication_services'
            'consumer_durables_and_apparel'
            'consumer_services'
            'transportation'
            'diversified_financials'
            'utilities'
            'capital_goods'
            'insurance'
            'chemicals'
            'metals_and_mining'
            'commercial_and_professional_services'
            'containers_and_packaging'
            'energy_equipment_and_services'
            'construction_materials'
            'paper_and_forest_products'
    
    -Returns:
        stonks (pandas DataFrame): list of selected tickers' price data
    '''
    all_stonks = read_stonk_data(date_from, date_to, data_dir=data_dir, data_prefix=data_prefix)
    
    if industries is None or not industries:
        return all_stonks
    else: 
        all_tickers = get_tickers_by_industry(industries=None, data_dir=data_dir, filename=stonk_list_filename)
        all_stonks = all_stonks.join(all_tickers, how='inner')
        return all_stonks[all_stonks['subindustry'].isin(industries)].drop(columns='subindustry')

### Quick examples

In [97]:
stonks = get_stonk_data_by_industry('2018-03-05', '2022-03-03')

In [98]:
stonks

,2018-03-05,2018-03-06,2018-03-07,2018-03-08,2018-03-09,2018-03-12,2018-03-13,2018-03-14,2018-03-15,2018-03-16,...,2022-02-16,2022-02-17,2022-02-18,2022-02-22,2022-02-23,2022-02-24,2022-02-25,2022-02-28,2022-03-01,2022-03-02
AAPL,42.40,42.36,41.97,42.43,43.15,43.57,43.15,42.79,42.84,42.68,...,172.55,168.88,167.30,164.32,160.07,162.74,164.85,165.12,163.20,166.56
ACIW,24.49,25.42,25.64,25.53,26.02,26.03,26.01,26.00,26.14,25.97,...,35.12,33.85,33.06,33.20,32.66,33.58,34.29,33.52,31.69,32.55
ADBE,211.16,212.01,216.86,216.40,221.11,220.94,219.76,218.44,218.87,225.55,...,477.70,457.71,442.56,438.40,429.45,463.82,465.54,467.68,466.68,471.18
ADP,107.29,107.05,106.84,109.09,110.83,109.62,109.49,108.29,108.64,109.28,...,204.24,200.46,200.45,199.90,196.29,200.22,204.36,204.44,202.31,206.29
ADS,172.26,174.73,174.95,175.53,177.78,176.94,172.90,171.19,169.64,170.27,...,72.82,70.38,70.50,68.69,66.93,64.89,67.65,67.45,63.50,64.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XRX,25.59,25.38,25.20,25.20,25.28,25.08,26.48,25.98,26.23,26.27,...,23.18,22.07,21.82,20.92,20.05,19.78,20.06,19.71,18.81,19.65
YEXT,13.00,12.94,13.00,13.53,12.50,12.46,12.20,12.37,12.34,12.25,...,7.98,7.63,7.39,7.30,7.13,7.43,7.43,7.42,7.20,7.25
ZBRA,141.02,143.22,143.98,141.40,141.18,144.75,144.20,146.79,144.40,143.34,...,430.97,423.50,420.31,409.76,399.29,415.33,418.07,413.34,405.58,415.27
ZEN,45.05,44.42,45.30,46.47,45.94,47.78,46.04,48.73,47.60,46.92,...,118.00,116.65,115.06,111.39,112.06,115.06,115.48,116.67,116.80,124.19


In [106]:
stonks = get_stonk_data_by_industry('2018-03-05', '2022-03-03', industries=['technology_hardware_and_equipment', 'software_and_services'])

In [116]:
stonks

,2018-03-05,2018-03-06,2018-03-07,2018-03-08,2018-03-09,2018-03-12,2018-03-13,2018-03-14,2018-03-15,2018-03-16,...,2022-02-16,2022-02-17,2022-02-18,2022-02-22,2022-02-23,2022-02-24,2022-02-25,2022-02-28,2022-03-01,2022-03-02
AAPL,42.40,42.36,41.97,42.43,43.15,43.57,43.15,42.79,42.84,42.68,...,172.55,168.88,167.30,164.32,160.07,162.74,164.85,165.12,163.20,166.56
ACIW,24.49,25.42,25.64,25.53,26.02,26.03,26.01,26.00,26.14,25.97,...,35.12,33.85,33.06,33.20,32.66,33.58,34.29,33.52,31.69,32.55
ADBE,211.16,212.01,216.86,216.40,221.11,220.94,219.76,218.44,218.87,225.55,...,477.70,457.71,442.56,438.40,429.45,463.82,465.54,467.68,466.68,471.18
ADP,107.29,107.05,106.84,109.09,110.83,109.62,109.49,108.29,108.64,109.28,...,204.24,200.46,200.45,199.90,196.29,200.22,204.36,204.44,202.31,206.29
ADS,172.26,174.73,174.95,175.53,177.78,176.94,172.90,171.19,169.64,170.27,...,72.82,70.38,70.50,68.69,66.93,64.89,67.65,67.45,63.50,64.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XRX,25.59,25.38,25.20,25.20,25.28,25.08,26.48,25.98,26.23,26.27,...,23.18,22.07,21.82,20.92,20.05,19.78,20.06,19.71,18.81,19.65
YEXT,13.00,12.94,13.00,13.53,12.50,12.46,12.20,12.37,12.34,12.25,...,7.98,7.63,7.39,7.30,7.13,7.43,7.43,7.42,7.20,7.25
ZBRA,141.02,143.22,143.98,141.40,141.18,144.75,144.20,146.79,144.40,143.34,...,430.97,423.50,420.31,409.76,399.29,415.33,418.07,413.34,405.58,415.27
ZEN,45.05,44.42,45.30,46.47,45.94,47.78,46.04,48.73,47.60,46.92,...,118.00,116.65,115.06,111.39,112.06,115.06,115.48,116.67,116.80,124.19


In [132]:
def combine_stonk_pairs(stonks_prices):
    # All ticker names must be unique
    assert all(stonks_prices.index.unique() == stonks_prices.index)
    
    combs = np.asarray(list(combinations(stonks_prices.index.unique(), 2)))
    
    return stonks_prices.loc[combs[:, 0]], stonks_prices.loc[combs[:, 1]]

In [146]:
stonks

,2018-03-05,2018-03-06,2018-03-07,2018-03-08,2018-03-09,2018-03-12,2018-03-13,2018-03-14,2018-03-15,2018-03-16,...,2022-02-16,2022-02-17,2022-02-18,2022-02-22,2022-02-23,2022-02-24,2022-02-25,2022-02-28,2022-03-01,2022-03-02
AAPL,42.40,42.36,41.97,42.43,43.15,43.57,43.15,42.79,42.84,42.68,...,172.55,168.88,167.30,164.32,160.07,162.74,164.85,165.12,163.20,166.56
ACIW,24.49,25.42,25.64,25.53,26.02,26.03,26.01,26.00,26.14,25.97,...,35.12,33.85,33.06,33.20,32.66,33.58,34.29,33.52,31.69,32.55
ADBE,211.16,212.01,216.86,216.40,221.11,220.94,219.76,218.44,218.87,225.55,...,477.70,457.71,442.56,438.40,429.45,463.82,465.54,467.68,466.68,471.18
ADP,107.29,107.05,106.84,109.09,110.83,109.62,109.49,108.29,108.64,109.28,...,204.24,200.46,200.45,199.90,196.29,200.22,204.36,204.44,202.31,206.29
ADS,172.26,174.73,174.95,175.53,177.78,176.94,172.90,171.19,169.64,170.27,...,72.82,70.38,70.50,68.69,66.93,64.89,67.65,67.45,63.50,64.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XRX,25.59,25.38,25.20,25.20,25.28,25.08,26.48,25.98,26.23,26.27,...,23.18,22.07,21.82,20.92,20.05,19.78,20.06,19.71,18.81,19.65
YEXT,13.00,12.94,13.00,13.53,12.50,12.46,12.20,12.37,12.34,12.25,...,7.98,7.63,7.39,7.30,7.13,7.43,7.43,7.42,7.20,7.25
ZBRA,141.02,143.22,143.98,141.40,141.18,144.75,144.20,146.79,144.40,143.34,...,430.97,423.50,420.31,409.76,399.29,415.33,418.07,413.34,405.58,415.27
ZEN,45.05,44.42,45.30,46.47,45.94,47.78,46.04,48.73,47.60,46.92,...,118.00,116.65,115.06,111.39,112.06,115.06,115.48,116.67,116.80,124.19


In [133]:
X, Y = combine_stonk_pairs(stonks)

In [139]:
X

,2018-03-05,2018-03-06,2018-03-07,2018-03-08,2018-03-09,2018-03-12,2018-03-13,2018-03-14,2018-03-15,2018-03-16,...,2022-02-16,2022-02-17,2022-02-18,2022-02-22,2022-02-23,2022-02-24,2022-02-25,2022-02-28,2022-03-01,2022-03-02
AAPL,42.40,42.36,41.97,42.43,43.15,43.57,43.15,42.79,42.84,42.68,...,172.55,168.88,167.30,164.32,160.07,162.74,164.85,165.12,163.20,166.56
AAPL,42.40,42.36,41.97,42.43,43.15,43.57,43.15,42.79,42.84,42.68,...,172.55,168.88,167.30,164.32,160.07,162.74,164.85,165.12,163.20,166.56
AAPL,42.40,42.36,41.97,42.43,43.15,43.57,43.15,42.79,42.84,42.68,...,172.55,168.88,167.30,164.32,160.07,162.74,164.85,165.12,163.20,166.56
AAPL,42.40,42.36,41.97,42.43,43.15,43.57,43.15,42.79,42.84,42.68,...,172.55,168.88,167.30,164.32,160.07,162.74,164.85,165.12,163.20,166.56
AAPL,42.40,42.36,41.97,42.43,43.15,43.57,43.15,42.79,42.84,42.68,...,172.55,168.88,167.30,164.32,160.07,162.74,164.85,165.12,163.20,166.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YEXT,13.00,12.94,13.00,13.53,12.50,12.46,12.20,12.37,12.34,12.25,...,7.98,7.63,7.39,7.30,7.13,7.43,7.43,7.42,7.20,7.25
YEXT,13.00,12.94,13.00,13.53,12.50,12.46,12.20,12.37,12.34,12.25,...,7.98,7.63,7.39,7.30,7.13,7.43,7.43,7.42,7.20,7.25
ZBRA,141.02,143.22,143.98,141.40,141.18,144.75,144.20,146.79,144.40,143.34,...,430.97,423.50,420.31,409.76,399.29,415.33,418.07,413.34,405.58,415.27
ZBRA,141.02,143.22,143.98,141.40,141.18,144.75,144.20,146.79,144.40,143.34,...,430.97,423.50,420.31,409.76,399.29,415.33,418.07,413.34,405.58,415.27


In [159]:
Y.iloc[0, -10:]

2022-02-16    35.12
2022-02-17    33.85
2022-02-18    33.06
2022-02-22    33.20
2022-02-23    32.66
2022-02-24    33.58
2022-02-25    34.29
2022-02-28    33.52
2022-03-01    31.69
2022-03-02    32.55
Name: ACIW, dtype: float64

In [141]:
res_roll, betas_roll, preds_roll = get_rolling_residuals(X, Y, l_reg=3, l_roll=1, dt=5)

In [151]:
betas_roll

array([0.06944337, 0.0712502 , 0.07263951, ..., 2.24156477, 2.24318449,
       2.24511534])

In [160]:
preds_roll.shape

(830790, 750)

-6.032812140740277e-08

In [28]:
# t1_fast = time.time()
# res, betas, preds  = get_rolling_residuals(X, Y, l_reg=2, l_roll=1, dt=5)
# t2_fast = time.time()

# t1_slow = time.time()
# res_slow, preds_slow = get_rolling_slow_residuals(X, Y, l_reg=2, l_roll=1, dt=5)
# t2_slow = time.time()

# print("Time slow: " + str(t2_slow-t1_slow))
# print("Time fast: " + str(t2_fast-t1_fast))

In [29]:
# t1_fast = time.time()
# res, preds, betas = get_residuals_many(X, Y)
# t2_fast = time.time()

# t1_slow = time.time()
# res_slow, preds_slow, betas_slow = get_slow_residuals_many(X, Y)
# t2_slow = time.time()

# print("Time slow: " + str(t2_slow-t1_slow))
# print("Time fast: " + str(t2_fast-t1_fast))

## Linear regression residuals

In [153]:
def get_residuals_many(X, Y):
    '''
    Vectorized calculation of residuals from many univariate linear regressions.
        Args:
        - X (numpy array of shape (n_pairs, d_time)): matrix of LR inputs X, each row represents a different regression, corresponding to the same rows in Y
        - Y (numpy array of shape (n_pairs, d_time)): matrix of LR inputs Y, each row represents a different regression, corresponding to the same rows in X
        Returns:
        - residuals (numpy array of shape (n_pairs, d_time)): matrix of resulting residuals between vectorized pairs of X and Y
        - betas (numpy array of shape (n_pairs, 1)): beta coefficients for each linear regression
        - Y_hat (numpy array of shape (n_pairs, d_time)): predictions using X
    '''
    # Stack 2D matrices into 3D matrices
    X = X.reshape(np.shape(X)[0], np.shape(X)[1], -1)
    Y = Y.reshape(np.shape(Y)[0], np.shape(Y)[1], -1)
    
    # Add bias/intercept in the form (Xi, 1)
    Z = np.concatenate([X, np.ones((np.shape(X)[0], np.shape(X)[1], 1))], axis=2)
    
    # Save the transpose as it's used a couple of times
    Z_t = Z.transpose(0, 2, 1)
    
    # Linear Regression equation solutions w.r.t. weight matrix
    # W contains (beta_coef, a_intercept) for each regression
    W = np.matmul(np.linalg.inv(np.matmul(Z_t, Z)),  np.matmul(Z_t, Y))
    
    # Predictions and residuals
    # Y_hat = np.matmul(Z, W).round(2)
    residuals = (Y - np.matmul(Z, W).round(2))
    
    # Y_hat returned for debugging purposes
    # return (residuals[:, :, 0], W[:, 0, 0], Y_hat[:, :, 0])
    return (residuals[:, :, 0], W[:, 0, 0])

In [102]:
def get_rolling_residuals(X, Y, l_reg, l_roll, dt):
    '''
    Calculates rolling window residuals in vectorized form. Returns the result as an array that repeats each ticker for the number of regressions calculated.
    For example, if the inputs are (Pair A, Pair B, Pair C) and l_roll / dt = 3, then the returned results will have the form as follows:
    (Pair A, Pair A, Pair A, Pair B, Pair B, Pair B, Pair C, Pair C, Pair C)
    Works best when l_reg and l_roll are integers.
        Args:
        - X (DataFrame of shape (n_pairs, >= l_reg + l_roll)): matrix of LR inputs X, each row representing not less than complete data period for rolling regressions (can be longer)
        - Y (DataFrame of shape (n_pairs, >= l_reg + l_roll)): matrix of LR inputs Y, each row representing not less than complete data period for rolling regressions (can be longer)
        - l_reg (float): length of each LR to calculate residuals, in years; will be multiplied by the adjusted number of days in a trading year
        - l_roll (float): length of rolling window, in years; will be multipled by the adjusted number of days in a trading year
        - dt (int): rolling window step size, in trading days; total trading year days will be reduced to be divisible by dt (by not more than the value of dt)
        Returns:
        - residuals (numpy array of shape (n_pairs * (l_roll/dt)+1, l_reg + l_roll)): matrix of resulting residuals between vectorized pairs of X and Y
        - betas (numpy array of shape (n_pairs * (l_roll/dt)+1, 1)): beta coefficients for each linear regression
        - Y_hat (numpy array of shape (n_pairs * (l_roll/dt)+1, l_reg + l_roll)): predictions using X
    '''
    # X = X.to_numpy().astype(np.float64)
    # Y = Y.to_numpy().astype(np.float64)
    
    _DAYS_IN_TRADING_YEAR = 252
    
    # Adjust days in a year so that the number is divisible by dt
    _DAYS_IN_TRADING_YEAR = _DAYS_IN_TRADING_YEAR - (_DAYS_IN_TRADING_YEAR % dt)
    l_reg_days = int(_DAYS_IN_TRADING_YEAR * l_reg)
    l_roll_days = int(_DAYS_IN_TRADING_YEAR * l_roll)
    total_days = l_reg_days + l_roll_days
    n_windows = (l_roll_days // dt) + 1
    n_x = X.shape[0]
    
    # Rolling window length must be divisible by dt
    assert (l_roll_days % dt) == 0
    
    # There has to be enough days' worth of data in X (and Y) and their shapes must match
    assert X.shape == Y.shape and X.shape[1] >= total_days
    
    # Take the total_days from the end of the arrays (most recent days first, oldest days at the end are cut off)
    X = X.iloc[:, -total_days:]
    Y = Y.iloc[:, -total_days:]
    
    # Create empty arrays that will contain windowed slices of our data
    X_windows = np.empty(shape=(n_x*n_windows, l_reg_days))
    Y_windows = np.empty(shape=(n_x*n_windows, l_reg_days))
    
    # Take windowed slices and place them into the created empty arrays
    for n in range(n_x):
        for i in range(n_windows):
            X_windows[(n*n_windows)+i] = X[n, i*dt:l_reg_days+(i*dt)].copy()
            Y_windows[(n*n_windows)+i] = Y[n, i*dt:l_reg_days+(i*dt)].copy()
    
    # Make sure we've got the windowing dimensions right
    assert X_windows.shape == (n_x*n_windows, l_reg_days) and Y_windows.shape == (n_x*n_windows, l_reg_days)
    
    # Sanity check
    assert all([
        X[0, -1] == X_windows[n_windows-1, -1],
        Y[0, -1] == Y_windows[n_windows-1, -1],
        X[-1, -1] == X_windows[-1, -1],
        Y[-1, -1] == Y_windows[-1, -1],
    ])
    
    # Calculate and return the residuals
    return get_residuals_many(X_windows, Y_windows)

### Slow residual functions (for testing)

In [103]:
# def get_rolling_slow_residuals(X, Y, l_reg, l_roll, dt):
#     _DAYS_IN_TRADING_YEAR = (252) - (252 % dt)
#     l_reg_days = _DAYS_IN_TRADING_YEAR * l_reg
#     l_roll_days = _DAYS_IN_TRADING_YEAR * l_roll
#     total_days = l_reg_days + l_roll_days
#     n_windows = l_roll_days // dt
#     n_x = X.shape[0]
    
#     assert (l_roll_days % dt) == 0
#     assert X.shape[1] >= total_days and Y.shape[1] >= total_days
    
#     X = X[:, -total_days:]
#     Y = Y[:, -total_days:]
    
#     # First window
#     X_windows = np.empty(shape=(n_x*n_windows, l_reg_days))
#     Y_windows = np.empty(shape=(n_x*n_windows, l_reg_days))
    
#     for n in range(n_x):
#         for i in range(n_windows):
#             X_windows = np.concatenate(( X_windows, X[n, i*dt:l_reg_days+(i*dt)] ))
#             Y_windows = np.concatenate(( Y_windows, Y[n, i*dt:l_reg_days+(i*dt)] ))
    
#     assert X_windows.shape == (n_x*n_windows, l_reg_days) and Y_windows.shape == (n_x*n_windows, l_reg_days)
    
#     return get_slow_residuals_many(X_windows, Y_windows)

In [104]:
# def get_slow_residuals_many(X, Y, n_jobs=-1):
#     lr = LinearRegression(n_jobs=n_jobs, fit_intercept=True)
#     X = X.reshape((X.shape[0], X.shape[1], -1))
#     Y = Y.reshape((Y.shape[0], Y.shape[1], -1))
    
#     preds = []
#     res = []
#     betas = []
#     for i in range(X.shape[0]):
#         lr.fit(X[i], Y[i])
#         preds.append(lr.predict(X[i]).round(2))
#         res.append(Y[i]-preds[-1])
#         betas.append(lr.coef_[0][0])
#     return (np.asarray(res)[:,:,0], np.asarray(preds)[:,:,0], np.asarray(betas))

### Quick examples

## Stock list preprocessing

In [49]:
def preprocess_stock_list(raw_data_path='data/raw_stonk_list.xls', output_path='data/stonk_list.csv'):
    '''
    Parses a raw excel file from CapitalIQ containing ticker names and their subindustries, validates
    unusual ticker names with Yahoo Finance, saving the processed data in CSV format.

        Parameters:
            Required:
                raw_data_path (string):
                    Path to the raw excel file.
                output_path (string):
                    Path where to save the parsed data.
                
        Returns:
            Nothing
    '''
    
    df = pd.read_excel(io=raw_data_path)
    
    # Drop NA rows
    df.dropna(axis=0, inplace=True)
    
    # Reset index and drop the first row
    df.reset_index(inplace=True, drop=True)
    df.drop(index=0, axis=0, inplace=True)
    
    # Drop unwanted columns
    df.drop(columns=df.columns[[1, 2, 3, 4, 5, 7, 8, 9]], inplace=True)
    
    # Rename remaining columns
    df.columns = ['ticker', 'subindustry']
    
    # Remove the '(Primary)' tag from subindustries
    df['subindustry'] = df['subindustry'].str.replace(r' \(Primary\)', '')
    
    # Remove everything until (and including) the semicolon for tickers
    df['ticker'] = df['ticker'].str.replace(r'(.*:)', '')
    
    df['ticker'] = df['ticker'].str.replace(r' WI', '.VI')
    df['ticker'] = df['ticker'].str.replace(r'\.WI', '.VI')
    
    # Replace the ticker endings for a Yahoo finance supported format
    df['ticker'] = df['ticker'].str.replace(r'\.PR', '-P')
    # df['ticker'] = df['ticker'].str.replace(r' PR', '-P')
    
    # Take all remaining tickers that have a dot
    dotted = df[df['ticker'].str.fullmatch(r'[A-Z]*\.[A-Z]')]
    
    # Replace the dots with dashes
    dashed = dotted.copy()
    dashed['ticker'] = dashed['ticker'].str.replace(r'\.', '-')
    
    # Remove the dots
    undotted = dotted.copy()
    undotted['ticker'] = undotted['ticker'].str.replace(r'\.', '')

    # Combine all variantas together
    all_variants = pd.concat([dotted, dashed, undotted])
    
    # Run all of these through Yahoo finance, get last day's price
    stonks = yf.download(list(all_variants['ticker'].astype('string').values), period='1m', interval='1d', group_by='column')
    
    # Drop all NA tickers (that failed to download)
    valid_tickers = stonks['Adj Close'].iloc[-1].dropna(axis=0).to_frame().reset_index()
    
    # Rename columns
    valid_tickers.columns = ['ticker', 'price']
    
    # Add subindustries to the remaining valid tickers
    valid_tickers = valid_tickers.join(all_variants.set_index('ticker'), on='ticker')
    
    # Drop the price column
    valid_tickers.drop(columns=valid_tickers.columns[[1]], inplace=True)
    
    # Remove all tickers that have a dot from main dataframe
    df = df[~df['ticker'].str.fullmatch(r'[A-Z]*\.[A-Z]')]
    
    # Add the validated tickers back
    df = pd.concat([df, valid_tickers], axis=0, ignore_index=True)
    
    # Make the subindustry strings more code friendly
    df['subindustry'] = df['subindustry'].str.replace(' ', '_')
    df['subindustry'] = df['subindustry'].str.lower()
    df['subindustry'] = df['subindustry'].str.replace(',', '')
    
    df.to_csv(path_or_buf=output_path, header=True, index=False)